In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.version

'3.6.3 (default, Mar  6 2020, 14:15:08) \n[GCC 5.4.0 20160609]'

In [3]:
import numpy as np
import cvxpy as cp

import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.optim as optim
import os
import random

import sys
sys.path.insert(0, './mlopt-micp')
sys.path.insert(0, './mlopt-micp/cartpole')

import optimizer
from problem import Cartpole
from src.ae import Encoder, get_cartpole_encoder

In [4]:
def euclidean_dist(x,y):
    # x: NxD
    # y: MxD
    n = x.size(0)
    m = y.size(0)
    d = x.size(1)
    assert d == y.size(1)
    
    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)
    return torch.pow(x-y, 2).sum(2)

In [5]:
pp = Cartpole()

In [7]:
print('Total number of classes: {}'.format(pp.n_strategies))
print('Length of feature vector: {}'.format(pp.n_features))

Total number of classes: 581
Length of feature vector: 13


In [8]:
dim_in, dim_z = pp.n_features, 4

enc = get_cartpole_encoder(dim_in, dim_z)
enc(torch.from_numpy(pp.features[:2]).float())

tensor([[0.0000, 0.0861, 0.0449, 0.0254],
        [0.0000, 0.0690, 0.1079, 0.0422]], grad_fn=<ReluBackward0>)

In [9]:
enc_dict = {}
str_dict = {}
encodings = enc(torch.from_numpy(pp.features).float())

for ii in range(len(pp.features)):
    str_idx = int(pp.labels[ii,0])
    str_dict[ii] = str_idx
    if str_idx in enc_dict.keys():
        enc_dict[str_idx] += [ii]
    else:
        enc_dict[str_idx] = [ii]

In [11]:
# training parameters
TRAINING_ITERATIONS = int(5000)
BATCH_SIZE = int(64)
CHECKPOINT_AFTER = int(1250)
SAVEPOINT_AFTER = int(2500)

rand_idx = list(np.arange(0, pp.n_strategies-1))

indices = [rand_idx[ii * BATCH_SIZE:(ii + 1) * BATCH_SIZE] for ii in range((len(rand_idx) + BATCH_SIZE - 1) // BATCH_SIZE)]
random.shuffle(indices)

optimizer = optim.Adam(enc.parameters())

In [12]:
N = pp.n_strategies # number of classes in training set
Nc = 10  # number of classes per episode
Ns = 15  # number of support examples per class
Nq = 8  # number of query examples per class

V = np.random.randint(0, pp.n_strategies, Nc)
Sk = {}  # support examples
Qk = {}  # query examples
ck = torch.zeros((Nc, dim_z))

for ii, v in enumerate(V):
    if len(enc_dict[v]) < Ns:
        Sk[v] = enc_dict[v]
        Qk[v] = enc_dict[v]
    else:
        Sk[v] = random.sample(enc_dict[v], Ns)
        Qk[v] = [kk for kk in enc_dict[v] if kk not in Sk[v]]
        if len(Qk[v]) > Nq:
            Qk[v] = random.sample(Qk[v], Nq)
    ck[ii] = torch.mean(encodings[Sk[v]], axis=0)

loss = 0.
for ii, v in enumerate(V):
    local_loss = 0.
    fx = encodings[Qk[v]]
    centr_dist = (fx - ck[ii]).pow(2).sum(dim=0)
    

In [13]:
loss = 0.
for ii in range(len(V)):
    local_loss = 0.
    fx = encodings[Qk[v]]
    dists = euclidean_dist(fx, ck)
    log_p_y = dists[:,ii] + torch.log(torch.sum(torch.exp(-dists), axis=1))
    loss += log_p_y.mean()

loss.backward()
optimizer.step()

In [ ]:
itr = 0
for epoch in range(TRAINING_ITERATIONS):
    iter_count = 0
    for ii, idx in enumerate(indices):
        optimizer.zero_grad()
        action = torch.empty(NUM_DATA, dim_u)

        encodings_idx = enc(torch.from_numpy(pp.features[idx]).float())
        loss = compute_loss(encodings_idx, str_dict)

        loss.backward()
        optimizer.step()

        if itr % CHECKPOINT_AFTER == 0:
            print('Avg. loss: {}'.format(loss.item()))

        if itr % SAVEPOINT_AFTER == 0:
            torch.save(model.state_dict(), fn_pt_model)

        iter_count += 1
        itr += 1